# 딥러닝 레이어의 이해, Embedding, Recurrent

리니어 레이어와 컨볼루션 레이어에 대해 살펴보았다. 이제 가중치 weight 개념을 이해 했다면, Embedding 레이어와 RNN 레이어와 LSTM의 동작방식에 대해 배워볼 차례이다.

## 희소 표현
[사과, 바나나, 배]를 표현할 때, [0,1,2]과 같이 단어를 매핑하거나, 둥글다, 색상으로 [[0,0],[1,1],[0,1]]처럼 특정 차원에 단어 혹은 의미를 매핑하는 방식을 희소표현이라 한다.

## 단어의 분산표현
모든 단어를 고정차원(e.g 256차원)의 백터로 표현하면 어떨까? 차원이 의미나 단어를 의미하지 않는다고 하고, 유사한 맥락만 보고 그러한 맥락의 단어가 의미도 비슷할 것이라는 분포가설에 따라 표현하는 것이다.

'나는', '을 먹는다' 사이에 들어가는 단어들은 유사한 의미를 갖겠죠? 그렇게 하면 유사한 단어들의 백터 사이의 거리를 가깝게 하고 그렇지 않은 단어들끼리 멀어지도록 조정하면서 단어를 백터로 표현하는 것이 분산표현이다.

## Embedding 레이어
분산 표현을 구현기 위해 n개의 단어 k차원으로 표현하는 n*k 형태의 분산 표현 사전을 만들고, 가중치의 파라미터 값을 학습을 통해 찾아가도록 한다.

## Embedding 레이어 학습 방법
임베딩 레이어는 케라스 신경망을 통해서 학습된다. 케라스에서 손실함수를 줄여주는 방식으로 학습할 수 있으며, 콘볼루션 레이어 등과 다른 점은, 입력값이 수치적인 의미를 갖지 않고 인덱스와 같은 의미라서, 결과 값이 입력값의 수학적인 계산이 의미를 갖지 않는다. 그래서 케라스의 임베딩 레이어는 word2vec과 같은 방식으로 동작한다고 볼 수 없다. word2vec은 단어들의 문맥을 잡기위한 임베딩 학습을 위한 시도하는 특별한 신경망 구축이 필요하다.
케라스 임베딩 레이어는 손실함수를 줄이기 위해서 학습한다. 감정분석을 한다고 보면, 임베딩 학습이 완벽한 단어 의미를 찾고, 문맥을 잡는다기 보다는 그들의 감정의 양극성을 확인하는 형태로 임베딩 레이어를 학습한다.

임베딩 레이어를 학습하기 위해서는 word2vec, skip_gram, Glove or CBOW 등이 활용된다.

## Embedding 레이어 톺아보기

간단하게 정리하면 컴퓨터용 단어사전이다. 단어를 n개 쓴다고 전달만 하면 컴퓨터가 알아서 사전을 만들고 분산표현을 차근차근 없데이트 합니다.

## 단어 간 유사도를 파악하는 방법
0,1만 인식하는 컴퓨터에게 자연어를 처리하도록 하는 자연어 처리, NLP는 어떻게 텍스트에서 유의미한 데이터를 추출해서 처리하는 것일까?

## 1. 가장 간단하고 원시적인 원-핫 인코딩
N개의 단어를 N차원의 백터로 표현하는 것이다. 가장 정확하게 분류가 가능하지만, 단어의 크기에따라 차원이 너무 커지고, 일정수준이 넘어가면 분류기 성능이 되려 0으로 수렴한다는 차원의 저주도 고려해야한다. 또한, 이방식은 단어들의 유사도를 파악하지는 못한다.

In [1]:
import tensorflow as tf

vocab = {      # 사용할 단어 사전 정의
    "i": 0,
    "need": 1,
    "some": 2,
    "more": 3,
    "coffee": 4,
    "cake": 5,
    "cat": 6,
    "dog": 7
}

sentence = "i i i i need some more coffee coffee coffee"
# 위 sentence
_input = [vocab[w] for w in sentence.split()]  # [0, 0, 0, 0, 1, 2, 3, 4, 4, 4]

vocab_size = len(vocab)   # 8

one_hot = tf.one_hot(_input, vocab_size)
print(one_hot.numpy())    # 원-핫 인코딩 벡터를 출력해 봅시다.

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]


2022-10-11 13:12:39.612500: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-11 13:12:39.612919: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


원핫인코딩은 이해하기 어렵지 않게 잘 구분되어 입력되어 있다.

## 2. 단어 임베딩, word embedding
'분산 표현'의 개념을 차용해서 단어를 임베딩하는 것이 시도 되었는데, 주어 + [][] + '공부했다' 라는 문장처럼 사이에 '수학', '과학'과 같은 단어들이 들어갈 수 있다면 유사성을 유추하는 방법이다. 즉 임베딩레이어를 통해 연산이 되면, 유사도가 나온다. 

In [3]:
distribution_size = 2   # 보기 좋게 2차원으로 분산 표현하도록 하죠!
linear = tf.keras.layers.Dense(units=distribution_size, use_bias=False)
one_hot_linear = linear(one_hot)

print("Linear Weight")
print(linear.weights[0].numpy())

print("\nOne-Hot Linear Result")
print(one_hot_linear.numpy())

Linear Weight
[[-0.77032435  0.20424563]
 [-0.05968374  0.36324823]
 [-0.46036702  0.35536027]
 [ 0.29822004 -0.19714737]
 [-0.6681304   0.07589316]
 [-0.6337753  -0.57273227]
 [ 0.06780905  0.06207407]
 [-0.7113961  -0.00132269]]

One-Hot Linear Result
[[-0.77032435  0.20424563]
 [-0.77032435  0.20424563]
 [-0.77032435  0.20424563]
 [-0.77032435  0.20424563]
 [-0.05968374  0.36324823]
 [-0.46036702  0.35536027]
 [ 0.29822004 -0.19714737]
 [-0.6681304   0.07589316]
 [-0.6681304   0.07589316]
 [-0.6681304   0.07589316]]


In [4]:
some_words = tf.constant([[3, 57, 35]])
# 3번 단어 / 57번 단어 / 35번 단어로 이루어진 한 문장입니다.

print("Embedding을 진행할 문장:", some_words.shape)
embedding_layer = tf.keras.layers.Embedding(input_dim=64, output_dim=100)
# 총 64개의 단어를 포함한 Embedding 레이어를 선언할 것이고,
# 각 단어는 100차원으로 분산 표현 할 것입니다.

print("Embedding된 문장:", embedding_layer(some_words).shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

Embedding을 진행할 문장: (1, 3)
Embedding된 문장: (1, 3, 100)
Embedding Layer의 Weight 형태: (64, 100)


## 이전정보를 반영하는 신경망, RNN

문장이나 영상, 음성의 데이터는 이미지와 다르계 순차적인 특성을 갖고 있다.

## 신경망이 이전정보를 반영하는 방법

이전 정보를 반영하는 두 가지 방법론
- (input , prev_hidden) -> hidden -> output
  - 이전에 전부다 기억하는 방식
- (input , prev_input) -> hidden -> output
  - 바로 직전만을 기억하는 방식

텐서플로우 프레임워크를 이용해 RNN을 만들어보자.

In [6]:
sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

print("RNN에 입력할 문장:", sentence)

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

print("Embedding을 위해 단어 매핑:", sentence_tensor.numpy())
print("입력 문장 데이터 형태:", sentence_tensor.shape)

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("\nEmbedding 결과:", emb_out.shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

rnn_seq_layer = \
tf.keras.layers.SimpleRNN(units=64, return_sequences=True, use_bias=False)
rnn_seq_out = rnn_seq_layer(emb_out)

print("\nRNN 결과 (모든 Step Output):", rnn_seq_out.shape)
print("RNN Layer의 Weight 형태:", rnn_seq_layer.weights[0].shape)

rnn_fin_layer = tf.keras.layers.SimpleRNN(units=64, use_bias=False)
rnn_fin_out = rnn_fin_layer(emb_out)

print("\nRNN 결과 (최종 Step Output):", rnn_fin_out.shape)
print("RNN Layer의 Weight 형태:", rnn_fin_layer.weights[0].shape)

RNN에 입력할 문장: What time is it ?
Embedding을 위해 단어 매핑: [[2 3 0 1 4]]
입력 문장 데이터 형태: (1, 5)

Embedding 결과: (1, 5, 100)
Embedding Layer의 Weight 형태: (5, 100)

RNN 결과 (모든 Step Output): (1, 5, 64)
RNN Layer의 Weight 형태: (100, 64)

RNN 결과 (최종 Step Output): (1, 64)
RNN Layer의 Weight 형태: (100, 64)


## LSTM 레이어 사용하기


RNN은 음성인식, 언어모델링 이미지 캡쳐 생성 등에는 의미있는 성능을 보여주긴 했으나, 간단한 RNN 구조는 한계도 가지고 있는데, 장기 의존성을 다루지 못한다. 입력데이터가 길어질수록 데이터 앞쪽의 정보가 뒤쪽까지 전달이 잘 안되는 현상이다. RNN의 은닉층을 학습하는 과정에서 기울기 소실, vanishing gradient이 발생하기 때문이다.

cell 이라는 메모리를 만들어서 은닉층에서 forget gate, input gate, gate gate, output gate를 통해 다음 순번의 은닉층에 연산이 되어지는 개념이다.

forget Gate Layer : cell state의 기존 정보를 얼마나 잊어버릴지를 결정하는 gate
input Gate Layer : 새롭게 들어온 정보를 기존 cell state에 얼마나 반영할지를 결정하는 gate
output Gate Layer : 새롭게 만들어진 cell state를 새로운 hidden state에 얼마나 반영할지를 결정하는 gate

LSTM의 변형을 주는 시도중에 GRU, Gated Recurrent Unit은 LSTM에서 cell state와 은닉층을 합치고, forget gate와 input gate를 통합했다.

### GRU

1. reset gate - 과거의 정보를 적당히 리셋시키는게 목적으로 sigmoid 함수를 출력해 은닉층에 곱해줍니다. LSTM에서 forget gate와 유사하다.

2. update gate - gate gate와 input gate를 합쳐놓은 느낌으로 과거와 현재의 정보를 최신화 비율을 결정한다.

3. candidate - 현 시점의 정보 후보군을 계산하는 단계

## 양방향(Bidirectional) RNN

'날이' '너무' [][][] '에어컨을' '켰다'와 같은 데이터가 있다면, 날이 너무 라는 앞 단어들로 추운지 더운지 예측하기 어렵다. 그렇지만 뒤에 에어컨을 켰다라고 하는 단어는 추운지 알 수 있다.

양방향 RNN은 문장 분석이나 생성보다는 주로 기계번역같은 테스크에 유리하다. 번역기를 만들때는 Transformer네트워크를 주로 사용한다.

In [8]:
import tensorflow as tf

sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("입력 문장 데이터 형태:", emb_out.shape)

bi_rnn = \
tf.keras.layers.Bidirectional(
    tf.keras.layers.SimpleRNN(units=64, use_bias=False, return_sequences=True)
)
bi_out = bi_rnn(emb_out)

print("Bidirectional RNN 결과 (최종 Step Output):", bi_out.shape)

입력 문장 데이터 형태: (1, 5, 100)
Bidirectional RNN 결과 (최종 Step Output): (1, 5, 128)


In [7]:
lstm_seq_layer = tf.keras.layers.LSTM(units=64, return_sequences=True, use_bias=False)
lstm_seq_out = lstm_seq_layer(emb_out)

print("\nLSTM 결과 (모든 Step Output):", lstm_seq_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_seq_layer.weights[0].shape)

lstm_fin_layer = tf.keras.layers.LSTM(units=64, use_bias=False)
lstm_fin_out = lstm_fin_layer(emb_out)

print("\nLSTM 결과 (최종 Step Output):", lstm_fin_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_fin_layer.weights[0].shape)


LSTM 결과 (모든 Step Output): (1, 5, 64)
LSTM Layer의 Weight 형태: (100, 256)

LSTM 결과 (최종 Step Output): (1, 64)
LSTM Layer의 Weight 형태: (100, 256)
